# 第3章 推荐系统冷启动问题

## 冷启动问题简介

冷启动问题主要分3类：  
1. **用户冷启动：**主要解决如何给新用户做个性化推荐的问题
2. **物品冷启动：**主要解决如何将新的物品推荐给可能对它感兴趣的用户
3. **系统冷启动：**主要解决如何在一个新开发的网站上（还没有用户，也没有用户行为，只有一些物品的信息）设计个性化推荐，从而在网站刚发布时就让用户体验到个性化推荐服务

针对2个不同的冷启动问题，有如下解决方案：  
1. 提供非个性化的推荐：最简单的例子是热门排行榜，等到用户数据收集到一定的时候，再切换为个性化推荐
2. 利用用户注册时提供的年龄、性别等数据做粗粒度的个性化
3. 利用用户的社交网络帐号登录（需要用户授权），导入用户在社交网站上的好友信息，然后给用户推荐其好友喜欢的物品（UserCF算法）
4. 要求用户在登录时对一些物品进行反馈，收集用户对这些物品的兴趣信息，然后给用户推荐那些和这些物品相似的物品（ItemCF算法和LFM算法）
5. 对于新加入的物品，可以利用内容信息，将它们推荐给喜欢过和它们相似的物品的用户（ItemCF算法）
6. 在系统冷启动时，引入专家的知识，通过一定的高效方式迅速建立起物品的相关度表（基于图的推荐算法，PersonalRank算法）

## 利用用户注册信息

用户的注册信息分3种：
1. **人口统计学信息：**包括用户的年龄、性别、职业、民族、学历和居住地
2. **用户兴趣的描述：**让用户用文字描述他们的兴趣
3. **从其他网站导入的用户站外行为数据：**比如用户通过微博的帐号登录，就可以在得到用户同意的情况下获取用户在第三方平台的一些行为数据和社交网络数据。

基于注册信息的个性化推荐流程基本如下：  
1. 获取用户的注册信息
2. 根据用户的注册信息对用户分类
3. 给用户推荐他所属分类中用户喜欢的物品

&emsp;&emsp;基于用户注册信息的推荐算法其**核心问题**是计算每种特征的用户喜欢的物品。对于每种特征$f$，计算具有这种特征的用户对各个物品的喜好程度$p(f, i)$。$p(f, i)$定义为物品$i$在具有$f$的特征的用户中的热门程度：$$p(f, i)=\left|N(i) \cap U(f)\right|$$其中$N(i)$是喜欢该物品的用户集合，$U(f)$是具有特征$f$的用户集合。  
&emsp;&emsp;为解决用户发现他们不容易发现的物品，将$p(f,i)$定义为喜欢物品$i$的用户中具有特征$f$的比例：$$p(f, i)=\frac{|N(i) \cap U(f)|}{|N(i)| + \alpha}$$其中参数$\alpha$的目的是解决数据稀疏的问题。

In [2]:
import os, sys
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold

from main.chapter3.age_most_popular import AgeMostPopular
from main.chapter3.country_most_popular import CountryMostPopular
from main.chapter3.demographic_most_popular import DemographicMostPopular
from main.chapter3.gender_most_popular import GenderMostPopular
from main.chapter3.most_popular import MostPopular
from main.util import lastfm_reader, metric

PROJECT_ROOT = os.path.dirname(sys.path[0])

In [1]:
data_path = os.path.join(PROJECT_ROOT, "src/data/lastfm-360K/usersha1-artmbid-artname-plays.csv")
profile_path = os.path.join(PROJECT_ROOT, "src/data/lastfm-360K/usersha1-profile.csv")

In [2]:
# 加载数据集
data, profile = lastfm_reader.load_data(data_path, profile_path)

D:\MyPythonWork\recommendation-system-practice-notes\src\main\util\lastfm_reader.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  profile['age'] = profile['age'].fillna(-1).astype(int)


In [3]:
def evaluate(model, train_dataset, test_dataset, N):
    """
    评估模型
    :param N: 推荐的商品个数
    :param K: 搜索邻近的用户个数
    :return: 精确率(precision), 召回率(recall)
    """
    recommens = model.recommend_users(test_dataset.keys(), N=N)
    all_items = lastfm_reader.get_all_items(train_dataset, test_dataset)

    recall = metric.recall(recommends=recommens, tests=test_dataset)
    precision = metric.precision(recommends=recommens, tests=test_dataset)
    coverage = metric.coverage(recommends=recommens, all_items=all_items)

    return precision, recall, coverage

def train(kf, data, popular_model):
    precisions = []
    recalls = []
    coverages = []
    for train_index, test_index in kf.split(data):
        test_dataset = data.iloc[test_index]
        train_dataset = data.iloc[train_index]
        train_dataset, test_dataset = lastfm_reader.convert_dict(train_dataset, test_dataset)
        # 模型训练
        model = popular_model(train_dataset, profile)
        model.fit()
        precision, recall, coverage = evaluate(model, train_dataset, test_dataset, N)
        precisions.append(precision)
        recalls.append(recall)
        coverages.append(coverage)
    return np.average(precisions), np.average(recalls), np.average(coverages)

In [4]:
# 对数据集进行划分
M = 10
N = 10
kf = KFold(n_splits=M, shuffle=True, random_state=1)
popular_group = [MostPopular, GenderMostPopular, AgeMostPopular, 
                 CountryMostPopular, DemographicMostPopular]
metric_value = []
index_names = []

for popular_model in popular_group:
    index_names.append(popular_model.__name__)
    metric_value.append(train(kf, data, popular_model))

In [5]:
pd.DataFrame(
    data=metric_value,
    index=index_names,
    columns=['Precision', 'Recall', 'Coverage']
)

,Precision,Recall,Coverage
MostPopular,0.034404,0.016758,0.000721
GenderMostPopular,0.035661,0.017370,0.001034
AgeMostPopular,0.005934,0.002891,0.006375
CountryMostPopular,0.038167,0.018699,0.023983
DemographicMostPopular,0.032707,0.016117,0.032876


## 选择合适的物品启动用户的兴趣

&emsp;&emsp;在新用户第一次访问推荐系统时，不立即给用户展示推荐系统，而是给用户提供一些物品，让用户反馈他们对这些物品的兴趣，然后根据用户反馈，提供个性化推荐。  

能够用来启动用户兴趣的物品具有以下特点：  
1. **比较热门：**让用户对热门的物品进行反馈
2. **具有代表性和区分性：**启动用户兴趣的物品不能是大众化或老少咸宜的，因为这样的物品对用户的兴趣没有区分新。
3. **启动物品集合需要有多样性：**在冷启动时，为了匹配多样的兴趣，需要提供具有很高覆盖率的启动物品集合，这些物品能覆盖几乎所有主流的用户兴趣。

&emsp;&emsp;首先，给定一群用户，用这群用户对物品评分的方差度量这群用户兴趣的一致程度。如果方差很小，说明这一群用户的兴趣不太一致，也就是说物品具有比较大的区分度，反之则说明，这群用户的兴趣比较一致。  
&emsp;&emsp;令$\sigma_{u \in U'}$为用户集合$U'$中所有评分的方差，通过如下方式度量一个物品的区分度$D(i)$：$$D(i)=\sigma_{u \in N^+(i)} + \sigma_{u \in N^-(i)} + \sigma_{u \in \bar{N}(i)}$$其中，$N^+(i)$是喜欢物品$i$的用户集合，$N^-(i)$是不喜欢物品$i$的用户集合，$\bar{N}(i)$是没有对物品$i$评分的用户集合，$\sigma_{u \in N^+(i)}$是喜欢物品$i$的用户对其他物品评分的方差，$\sigma_{u \in N^-(i)}$是不喜欢物品$i$的用户对其他物品评分的方差，$\sigma_{u \in \bar{N}(i)}$是没有对物品$i$评分的用户对其他物品的评分的方差。  

&emsp;&emsp;对于物品$i$，将用户分成3类——喜欢物品$i$的用户、不喜欢物品$i$的用户和不知道物品$i$的用户（即没有给$i$评分的用户）。如果这3类用户集合内的用户对其他的物品兴趣都不一致，说明物品$i$具有较高的区分度。

## 利用物品的内容信息

&emsp;&emsp;物品内容可以通过向量空间模型表示，该模型会将物品表示成一个关键词向量，对于中文，首先要对文本进行分词，将字流变成词流，然后从词流中检测出命名实体（如人名、地名、组织名等），这些实体和一些其他重要的词将组成关键词集合，最后对关键词进排名，计算每个关键词的权重，从而生成关键词向量。

将内容表示成一个关键词向量，对于物品$d$，关键词向量：$$d_i=\{(e_1,w_1),(e_2,w_2),\cdots\}$$其中，$e_i$就是关键词，$w_i$是关键词对应的权重。如果物品是文本，可以用信息检索领域TF-IDF公式计算词的权重：$$w_i=\frac{\text{TF}(e_i)}{\log \text{DF}(e_i)}$$  
在给定物品内容的关键词向量后，物品内容相似度可以通过向量之间的余弦相似度计算：$$w_{ij}=\frac{d_i \cdot d_j}{\sqrt{\|d_i\|\|d_j\|}}$$  

&emsp;&emsp;向量空间模型在内容数据丰富时，可以获得比较好的效果。以文本为例，如果是计算长文本的相似度，用向量空间末利用关键词计算相似度，已经可以获得很好的精确度。但是，如果文本很短，关键词很少，向量空间模型就很难计算出准确的相似度。

In [3]:
from main.chapter3.content_item_knn import ContentItemKNN
from main.util import metric, movielen_reader

PROJECT_ROOT = os.path.dirname(sys.path[0])

ratings_path = os.path.join(PROJECT_ROOT, "src/data/ml-1m/ratings.dat")
movies_path = os.path.join(PROJECT_ROOT, "src/data/ml-1m/movies.dat")

# 加载数据集
ratings_dataset = movielen_reader.load_data(ratings_path, ['user', 'item', 'rating', 'timestamp'])
movies_dataset = movielen_reader.load_data(movies_path, ['item', 'title', 'genres'])
movies_dataset['genres'] = movies_dataset['genres'].str.split('|')
ratings_dataset = ratings_dataset[['user', 'item', 'rating']]

contents = {row['item']: row['genres'] for _, row in movies_dataset.iterrows()}

In [4]:
M = 10
N = 10
K = 10
# K-Fold模型训练
kf = KFold(n_splits=M, shuffle=True, random_state=1)

In [5]:
def train_popularity(train_dataset):
    """计算训练集的流行度"""
    train_popularity = dict()
    for user, items in train_dataset.items():
        for item in items:
            train_popularity[item] = train_popularity.get(item, 0) + 1
    return train_popularity

def evaluate(model, train_dataset, test_dataset, N, K):
    """
    评估模型
    :param N: 推荐的商品个数
    :param K: 搜索邻近的用户个数
    :return: 精确率(precision), 召回率(recall), 覆盖率(coverage)
    """
    recommens = model.recommend_users(test_dataset.keys(), N=N, K=K)
    all_items = movielen_reader.get_all_items(train_dataset, test_dataset)
    item_popularity = train_popularity(train_dataset)

    recall = metric.recall(recommends=recommens, tests=test_dataset)
    precision = metric.precision(recommends=recommens, tests=test_dataset)
    coverage = metric.coverage(recommends=recommens, all_items=all_items)
    popularity = metric.popularity(item_popular=item_popularity, recommends=recommens)

    return precision, recall, coverage, popularity

def train(kf, data, contents, N, K):
    precisions = []
    recalls = []
    coverages = []
    popularitys = []
    for train_index, test_index in kf.split(data):
        test_dataset = data.iloc[test_index]
        train_dataset = data.iloc[train_index]
        train_dataset, test_dataset = 
            movielen_reader.convert_dict(train_dataset, test_dataset)
        # 模型训练
        model = ContentItemKNN(train_dataset, contents)
        model.fit()
        precision, recall, coverage, popularity = 
            evaluate(model, train_dataset, test_dataset, N, K)
        precisions.append(precision)
        recalls.append(recall)
        coverages.append(coverage)
        popularitys.append(popularity)
    return (np.average(precisions), np.average(recalls), 
            np.average(coverages), np.average(popularitys))

In [6]:
metric_value = [train(kf, ratings_dataset, contents, N, K)]

In [7]:
 # 得到指标
pd.DataFrame(
    data=metric_value,
    index=[ContentItemKNN.__name__],
    columns=['Precision', 'Recall', 'Coverage', 'Popularity']
)

,Precision,Recall,Coverage,Popularity
ContentItemKNN,0.009185,0.015411,0.168295,4.654557


## 发挥专家的作用

&emsp;&emsp;很多推荐系统在建立时，既没有用户的行为数据，也灭有充足的物品内容信息来计算准确的物品相似度。那么，为了在推荐系统建立时，就让用户得到比较好的体验，很多系统都利用专家进行标注。  
&emsp;&emsp;Jinni在电影基因工程中采用了半人工、半自动的方式：  
    1. 让专家对电影进行标注，每个电影多有大约50个基因，这些基因来自大约1000个基因库
    2. 使用NLP和机器学习，通过分析用户对电影的评论和电影的一些内容属性，对电影（特别是新电影）进行自己的标注
    3. 设计了让用户对基因进行反馈的界面

&emsp;&emsp;Jinni通过专家和机器学习相结合的方式，解决了系统冷启动的问题。